In [2]:

from pymongo import MongoClient
from tqdm import tqdm
import re
import pickle
import json



In [33]:
client = MongoClient() # intialize client

In [34]:
## Papers
all_records = client.predsynth.papers_v2.find()
client.predsynth.papers_v2.estimated_document_count() # Check number of papers in papers collection

5762732

In [1]:
# Define keywords for cement and concrete
include_keywords = ["cement", "concrete", "cementitious", "mortar", "binder"]


In [36]:
def filter_papers_by_criteria(records, include_keywords, exclude_keyword, min_abstract_length):
    filtered_records = []

    # Convert include_keywords to lowercase for case-insensitive matching
    include_keywords = [keyword.lower() for keyword in include_keywords]

    for record in tqdm(records, desc="Filtering records"):
        # Ensure that title and abstract are strings and convert them to lowercase
        title = str(record.get("title", "")).lower()
        abstract = str(record.get("abstract", "")).lower()

        # Check if keywords exist and are a list, then convert to lowercase
        record_keywords = record.get("keywords")
        if record_keywords is None:
            record_keywords = []
        else:
            record_keywords = [keyword.lower() for keyword in record_keywords]

        # Check title, keywords, and abstract for any of the include_keywords
        if (any(re.search(r'\b' + re.escape(keyword) + r'\b', title) for keyword in include_keywords) or
            any(keyword in record_keywords for keyword in include_keywords) or
            any(re.search(r'\b' + re.escape(keyword) + r'\b', abstract) for keyword in include_keywords)):
            # Exclude if it contains the exclude keyword
            if not re.search(r'\b' + re.escape(exclude_keyword) + r'\b', abstract):
                filtered_records.append(record)
    return filtered_records



# Keywords for title and the must-have keywords for abstract
exclude_keyword = 'dental'
min_abstract_length=20
# Apply the function
filtered_records = filter_papers_by_criteria(all_records, include_keywords, exclude_keyword, min_abstract_length)

Filtering records: 5762751it [50:02, 1919.49it/s]


In [39]:
# Count the number of papers
num_related_papers = len(filtered_records)
print(num_related_papers)

87529


In [41]:
from bson import ObjectId
import json

def json_serialize(obj):
    """
    JSON serializer for objects not serializable by default json code
    """
    if isinstance(obj, ObjectId):
        return str(obj)
    raise TypeError ("Type %s not serializable" % type(obj))
    
def json_deserialize(obj):
    # Implement deserialization if needed, e.g., converting string back to datetime
    return obj

In [42]:

# Usage
with open('filtered_records.json', 'w') as file:
    json.dump(filtered_records, file, default=json_serialize)